In [1]:
# autoreload modules
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import pandas as pd
import gnss_lib_py as glp
from ubx_parser import UbxParser
import matplotlib.pyplot as plt

In [127]:
# input UBX filepath
# UBX_FILENAME = "/Users/derekknowles/personal-data/ublox/short_test/ubx_full_messages_no_tm2.ubx"
# UBX_FILENAME = "/Users/derekknowles/personal-data/ublox/short_test/ubx_full_messages_no_tm2_round_2.ubx"
# UBX_FILENAME = "/Users/derekknowles/Library/CloudStorage/GoogleDrive-derek@hivemapper.com/Shared drives/bee-sensors-internal/gnss/u-blox-data/OCT2_ACTIVE_EMBARCADERO_DOWNTOWN.ubx"
# UBX_FILENAME = "/Users/derekknowles/Library/CloudStorage/GoogleDrive-derek@hivemapper.com/Shared drives/bee-sensors-internal/gnss/u-blox-data/OCT2_PASSIVE_EMBARCADERO_DOWNTOWN.ubx"
# UBX_FILENAME = "/Users/derekknowles/Library/CloudStorage/GoogleDrive-derek@hivemapper.com/Shared drives/bee-sensors-internal/gnss/u-blox-data/OCT2_ACTIVE_LAFAYETTE_HIGHWAYS.ubx"
UBX_FILENAME = "/Users/derekknowles/Library/CloudStorage/GoogleDrive-derek@hivemapper.com/Shared drives/bee-sensors-internal/gnss/u-blox-data/OCT2_PASSIVE_LAFAYETTE_HIGHWAYS.ubx"
# UBX_FILENAME = "/Users/derekknowles/Library/CloudStorage/GoogleDrive-derek@hivemapper.com/Shared drives/bee-sensors-internal/gnss/u-blox-data/OCT3_ACTIVE_PRESIDIO.ubx"
# UBX_FILENAME = "/Users/derekknowles/Library/CloudStorage/GoogleDrive-derek@hivemapper.com/Shared drives/bee-sensors-internal/gnss/u-blox-data/OCT3_PASSIVE_PRESIDIO.ubx"

# whether need to parse to CSV (only needs to happen once)
PARSE_TO_CSV = True

# Scale factor to shrink large data
# recommend 100 for files of about an hour
SCALE_FACTOR = 100

# MAXIMUM NUMBER OF SATELLITES SEEN
MAX_NUM_SATS = 50

# MAXIMUM C/N0 VALUE EXPECTED
MAX_CN0 = 50

In [128]:
if PARSE_TO_CSV:
    # parse UBX file to CSV
    UbxParser(UBX_FILENAME)

dir_name = os.path.join(os.path.dirname(os.path.abspath('')),
                                        "results",os.path.basename(UBX_FILENAME).split(".")[0])

## Basic Data Metrics

In [129]:
nav_pvt = glp.NavData(csv_path= os.path.join(dir_name, "NAV_PVT.csv"))

In [ ]:
start_time = glp.gps_millis_to_datetime(min(nav_pvt["gps_millis"]))
end_time = glp.gps_millis_to_datetime(max(nav_pvt["gps_millis"]))
log_duration = end_time - start_time

print("log start time: ", start_time)
print("log end time: ", end_time)
print("log duration: ", log_duration)

## Plot UBX PVT lat/lon solution

In [ ]:
glp.plot_map(nav_pvt, fname=os.path.join(dir_name, "map_lat_lon.png"), save=True)

## Plot Number of Satellites Seen vs. Used in Position Solution

In [132]:
nav_sat = glp.NavData(csv_path=os.path.join(dir_name, "NAV_SAT.csv"))

In [ ]:
times_all, all_counts = np.unique(nav_sat["gps_millis"], return_counts=True)
t0 = times_all[0]
times_all = [(t-t0)/(1000*60) for t in times_all]

# averaging window
N = SCALE_FACTOR*10


counts_all = glp.NavData()
counts_all["time_min"] = times_all
counts_all["sats_seen"] = all_counts

counts_all_avg = glp.NavData()
counts_all_avg["time_min"] = times_all[(N//2):-(N//2)]
counts_all_avg["sats_seen_avg"] = np.convolve(all_counts, np.ones((N,))/N, mode='valid')[:-1]

times_used, used_counts = np.unique(nav_sat.where("svUsed",1)["gps_millis"], return_counts=True)
t0 = times_used[0]
times_used = [(t-t0)/(1000*60) for t in times_used]
counts_used = glp.NavData()
counts_used["time_min"] = times_used
counts_used["sats_used"] = used_counts

counts_used_avg = glp.NavData()
counts_used_avg["time_min"] = times_used[(N//2):-(N//2)]
counts_used_avg["sats_used_avg"] = np.convolve(used_counts, np.ones((N,))/N, mode='valid')[:-1]

fig = glp.plot_metric(counts_all,"time_min","sats_seen",linestyle="None")
fig = glp.plot_metric(counts_all_avg,"time_min","sats_seen_avg",markersize=0,fig=fig,linewidth=3)
fig = glp.plot_metric(counts_used,"time_min","sats_used",linestyle="None",fig=fig)
fig = glp.plot_metric(counts_used_avg,"time_min","sats_used_avg",markersize=0,fig=fig,c='C5',linewidth=3)
# fig.set_ylim(0,MAX_NUM_SATS)
fig.axes[0].set_ylim(0,MAX_NUM_SATS)
plt.legend(["SVs Seen","Avg SVs Seen","SVs Used","Avg SVs Used"])

fig.savefig(os.path.join(dir_name, "sats_seen_vs_used.png"))

## Plot Constellation Count Tracked from NAV-SAT messages

In [134]:
def plot_stacked_constellations(nav_sat, used_only=False, scale_factor=SCALE_FACTOR):
    """ Plot stacked constellations

    Parameters
    ----------
    nav_sat : gnss_lib_py.NavData
        DataFrame containing NAV-SAT data
    used_only : bool
        Whether to plot only the used satellites
    scale_factor : int
        Use only one timestep out of every scale_factor timesteps
    """

    fig, ax = plt.subplots(1, 1, figsize=(5, 5))

    if used_only:
        nav_sat = nav_sat.where("svUsed",1)

    all_constellations = np.unique(nav_sat["gnss_id"])
    times = []
    unique_times = np.unique(nav_sat["gps_millis"])[::scale_factor]

    constellation_counts = {constellation : [] for constellation in all_constellations}
    for idx, timestamp in enumerate(unique_times):
        if idx % int(len(unique_times)/5.) == 0:
            print(f"processing {idx} of {len(unique_times)}")
        subset_idxs = np.argwhere(nav_sat["gps_millis"] == timestamp)
        for constellation in all_constellations:
            constellation_counts[constellation].append(np.count_nonzero(nav_sat["gnss_id",subset_idxs[:,0].tolist()] == constellation))

    print("all done computing")
    t0 = unique_times[0]
    times = [(t-t0)/(1000*60) for t in unique_times]
    labels = [c for c in glp.GNSS_ORDER if c in all_constellations]
    stackplot_data = [constellation_counts[c] for c in labels]


    ax.stackplot(times, stackplot_data, labels=labels)

    title = os.path.basename(UBX_FILENAME).split("_")[1] + " Antenna "
    if used_only:
        title += "Used in NAV solution"
    else:
        title += "All Sats Seen"
    ax.set_title(title)
    ax.set_ylabel("Number of Satellites")
    ax.set_xlabel("Time (min)")
    ax.legend(loc="upper right")
    ax.set_ylim(0,MAX_NUM_SATS)

    return fig

In [ ]:
fig = plot_stacked_constellations(nav_sat)
fig.savefig(os.path.join(dir_name, "sats_sat_tracked.png"))
fig = plot_stacked_constellations(nav_sat, used_only=True)
fig.savefig(os.path.join(dir_name, "sats_sat_tracked_used.png"))

## Plot all Satellites Searched For

In [136]:
nav_sig = glp.NavData(csv_path=os.path.join(dir_name, "NAV_SIG.csv"))

In [ ]:
fig = plot_stacked_constellations(nav_sig)
fig.savefig(os.path.join(dir_name, "sats_sig_searched.png"))

## Plot C/N0 across constellations

In [ ]:
cn0_avg = ("3)_c/n0_avg",[])
cn0_max = ("1)_c/n0_max",[])
cn0_min = ("4)_c/n0_mins",[])
cn0_top_4 = ("2)_c/n0_top_4",[])

nav_sat_used = nav_sat.where("svUsed",1)
unique_times = np.unique(nav_sat_used["gps_millis"])[::SCALE_FACTOR]
t0 = unique_times[0]
times = [(t-t0)/(1000*60) for t in unique_times]

for idx, timestamp in enumerate(unique_times):
    if idx % int(len(unique_times)/5.) == 0:
        print(f"processing {idx} of {len(unique_times)}")
    
    subset = nav_sat_used.where("gps_millis",timestamp)
    cn0 = subset["cn0_dbhz"]
    cn0_avg[1].append(np.mean(cn0))
    cn0_max[1].append(np.max(cn0))
    cn0_min[1].append(np.min(cn0))
    if len(subset) >= 4:
        cn0_top_4[1].append(np.mean(np.sort(cn0)[-4:]))
    else:
        cn0_top_4[1].append(np.mean(cn0))
all_cn0_data = glp.NavData()
for metric in [cn0_max, cn0_top_4, cn0_avg, cn0_min]:
    temp_data = glp.NavData()
    temp_data["time_min"] = times
    temp_data["metric"] = np.array([[metric[0]]*len(times)])
    temp_data["cn0_dbhz"] = metric[1]
    all_cn0_data = glp.concat(all_cn0_data, temp_data, axis=1)

fig = glp.plot_metric(all_cn0_data,"time_min","cn0_dbhz",groupby="metric",linestyle="None")


# averaging window
N = 10
cn0_avg_data = glp.NavData()
cn0_avg_data["time_min"] = times[(N//2):-(N//2)]
cn0_avg_data["cn0_dbhz"] = np.convolve(cn0_avg[1], np.ones((N,))/N, mode='valid')[:-1]
fig = glp.plot_metric(cn0_avg_data,"time_min","cn0_dbhz",
                      markersize=0,fig=fig,c='C5',linewidth=3, label="Rolling Avg")
plt.legend(title="Metric")
plt.ylim(0,MAX_CN0)

fig.savefig(os.path.join(dir_name,"CN0.png"))

In [139]:
nav_dop = glp.NavData(csv_path=os.path.join(dir_name, "NAV_DOP.csv"))

In [ ]:
fig = glp.plot_metric(nav_pvt,"gps_millis","hAcc",linestyle="None")
fig.savefig(os.path.join(dir_name, "hAcc_vs_gps_millis.png"))

In [ ]:
def plot_distribution(data, threshold, title='Distribution Plot', x_label='Value'):

    fig = plt.figure()
    # Create the CDF
    x = np.sort(data)
    y = np.arange(1, len(x) + 1) / len(x)
    plt.plot(x, y, label='CDF', color='C1')

    # the value for which you want to calculate the percentage below
    value = threshold

    # Calculate the percentage of values below the specified value
    percentage = np.sum(data <= value) / len(data) * 100

    # Add a vertical line at the specified value
    plt.axvline(x=value, color='C0', linestyle='--', label=f'Value: {value}')

    # Add a text annotation for the percentage
    plt.text(value, 0.8, f'   {percentage:.2f}% below {value}', color='C0')

    # Customize the plot
    plt.xlabel(x_label)
    plt.ylabel('Percentage')
    plt.title(title)

    return fig

fig = plot_distribution(nav_dop["hDOP"], 4.0, title="hDOP Distribution", x_label="hDOP")
fig.savefig(os.path.join(dir_name, "hDOP_distribution.png"))
fig = plot_distribution(nav_dop["gDOP"], 6.0, title="vDOP Distribution", x_label="vDOP")
fig.savefig(os.path.join(dir_name, "vDOP_distribution.png"))
fig = plot_distribution(nav_pvt["hAcc"]/1000., 1.0, title="hAcc Distribution", x_label="Horizontal Accuracy Estimate [m]")
fig.savefig(os.path.join(dir_name, "hAcc_distribution.png"))

## Plot Skyplot of Data

In [142]:
"""Visualization functions for GNSS data.

"""

__authors__ = "D. Knowles"
__date__ = "27 Jan 2022"

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb
from matplotlib.collections import LineCollection

from gnss_lib_py.visualizations.style import *
from gnss_lib_py.navdata.navdata import NavData
from gnss_lib_py.utils.coordinates import add_el_az

def plot_skyplot(navdata, receiver_state,
                 save=False, prefix="", fname=None,
                 add_sv_id_label=True, step = "auto", trim_options=None):
    """Skyplot of satellite positions relative to receiver.

    First adds ``el_sv_deg`` and ``az_sv_deg`` rows to navdata if they
    do not yet exist.

    Breaks up satellites by constellation names in ``gnss_id`` and the
    ``sv_id`` if the row is present in navdata.

    Parameters
    ----------
    navdata : gnss_lib_py.navdata.navdata.NavData
        Instance of the NavData class. Must include ``gps_millis`` as
        well as satellite ECEF positions as ``x_sv_m``, ``y_sv_m``,
        ``z_sv_m``, ``gnss_id`` and ``sv_id``.
    receiver_state : gnss_lib_py.navdata.navdata.NavData
        Either estimated or ground truth receiver position in ECEF frame
        in meters as an instance of the NavData class with the
        following rows: ``x_rx*_m``, ``y_rx*_m``, ``z_rx*_m``, ``gps_millis``.
    save : bool
        Saves figure if true to file specified by ``fname`` or defaults
        to the Results folder otherwise.
    prefix : string
        File prefix to add to filename.
    fname : string or path-like
        Path to save figure to. If not None, ``fname`` is passed
        directly to matplotlib's savefig fname parameter and prefix will
        be overwritten.
    add_sv_id_label : bool
        If the ``sv_id`` row is available, will add SV ID label near the
        satellite trail.
    step : int or "auto"
        Skyplot plotting is sped up by only plotting a portion of the
        satellite trajectories. If default is set to "auto" then it will
        plot a maximum of 50 points across each satellite trajectory. If
        the step variable is set to a positive integer ``n`` then only
        every nth point will be used in the trajectory. Setting the
        steps variable to 1 will plot every satellite trajectory point
        and may be slow to plot.
    trim_options : None or dict
        The ``trim_options`` variables gives control for line segments
        being trimmed between satellite points. For example, if 24 hours
        of a satellite is plotted, often the satellite will come in and
        out of view and the segment between when it was lost from view
        and when the satellite comes back in view should be trimmed.
        If trim_options is set to the default of None, then the default
        is set of trimming according to az_and_el and gps_millis. The
        current options for the trim_options dictionary are listed here.
        {"az" : az_limit} means that if at two timesteps the azimuth
        difference in degrees is greater than az_limit, then the line
        segment will be trimmed.
        {"az_and_el" : (az_limit,el_limit)} means that if at two
        timesteps the azimuth difference in degrees is greater than
        az_limit as well as the average of the elevation angle across
        the two timesteps is less than el_limit in degrees, then the
        line segment will be trimmed. The el_limit is because satellites
        near 90 degrees elevation can traverse large amounts of degrees
        in azimuth in a valid trajectory but at low elevations should
        not have such large azimuth changes quickly.
        {"gps_millis",gps_millis_limit} means that line segments will be
        trimmed if the milliseconds between the two points is larger
        than the gps_millis_limit. This option only works if the
        gps_millis row is included in the ``navdata`` variable input.
        Default options for the trim options are :code:`"az_and_el" : (15.,30.)`
        and :code:`"gps_millis" : 3.6E6`.

    Returns
    -------
    fig : matplotlib.pyplot.figure
        Figure object of skyplot.

    """

    if not isinstance(navdata,NavData):
        raise TypeError("first arg to plot_skyplot "\
                          + "must be a NavData object.")

    if not isinstance(prefix, str):
        raise TypeError("Prefix must be a string.")

    # add elevation and azimuth data.
    # add_el_az(navdata, receiver_state, inplace=True)

    # create new figure
    fig = plt.figure(figsize=(6,4.5))
    axes = fig.add_subplot(111, projection='polar')

    navdata = navdata.copy()
    navdata["az_sv_rad"] = np.radians(navdata["az_sv_deg"])
    # remove SVs below horizon
    navdata = navdata.where("el_sv_deg",0,"geq")
    # remove np.nan values caused by potentially faulty data
    navdata = navdata.where("az_sv_rad",np.nan,"neq")
    navdata = navdata.where("el_sv_deg",np.nan,"neq")

    for c_idx, constellation in enumerate(sort_gnss_ids(np.unique(navdata["gnss_id"]))):
        const_subset = navdata.where("gnss_id",constellation)
        color = "C" + str(c_idx % len(STANFORD_COLORS))
        cmap = new_cmap(to_rgb(color))
        marker = MARKERS[c_idx % len(MARKERS)]
        const_label_created = False

        # iterate through each satellite
        for sv_name in np.unique(const_subset["sv_id"]):
            sv_subset = const_subset.where("sv_id",sv_name)

            # only plot ~ 50 points for each sat to decrease time
            # it takes to plot these line collections if step == "auto"
            if isinstance(step,str) and step == "auto":
                step = max(1,int(len(sv_subset)/50.))
            elif isinstance(step, int):
                step = max(1,step)
            else:
                raise TypeError("step varaible must be 'auto' or int")
            points = np.array([np.atleast_1d(sv_subset["az_sv_rad"])[::step],
                               np.atleast_1d(sv_subset["el_sv_deg"])[::step]]).T
            points = np.reshape(points,(-1, 1, 2))
            segments = np.concatenate([points[:-1], points[1:]], axis=1)
            norm = plt.Normalize(0,len(segments))

            if trim_options is None:
                trim_options = {
                                "az_and_el" : (15.,30.),
                                "gps_millis" : 3.6E6,
                                }
            plotted_idxs = np.array([True] * len(segments))

            if "az" in trim_options and len(segments) > 2:
                # ignore segments that cross more than az_limit degrees
                # in azimuth between timesteps
                az_limit = np.radians(trim_options["az"])
                az_idxs = ~((np.abs(np.diff(np.unwrap(segments[:,:,0]))) >= az_limit)[:,0])
                plotted_idxs = np.bitwise_and(plotted_idxs, az_idxs)
            if "az_and_el" in trim_options and len(segments) > 2:
                # ignore segments that cross more than az_limit degrees
                # in azimuth between timesteps and are at an elevation
                # less than el_limit degrees.
                # These satellites are assumed to be the satellites
                # coming in and out of view in a later part of the orbit
                az_limit = np.radians(trim_options["az_and_el"][0])
                el_limit = trim_options["az_and_el"][1]
                az_and_el_idxs = ~(((np.abs(np.diff(np.unwrap(segments[:,:,0]))) >= az_limit)[:,0]) \
                                 & (np.mean(segments[:,:,1],axis=1) <= el_limit))
                plotted_idxs = np.bitwise_and(plotted_idxs, az_and_el_idxs)
            if "gps_millis" in trim_options and "gps_millis" in sv_subset.rows \
                and len(segments) > 2:
                # ignore segments if there is more than gps_millis_limit
                # milliseconds between the time segments
                gps_millis_limit = trim_options["gps_millis"]

                all_times = np.atleast_2d(sv_subset["gps_millis"][::step]).T
                point_times = np.concatenate([all_times[:-1],all_times[1:]],
                                              axis=1)
                gps_millis_idxs = (np.abs(np.diff(point_times)) <= gps_millis_limit)[:,0]
                plotted_idxs = np.bitwise_and(plotted_idxs, gps_millis_idxs)

            segments = segments[list(plotted_idxs)]

            local_coord = LineCollection(segments, cmap=cmap,
                            norm=norm, linewidths=(4,),
                            array = range(len(segments)))
            axes.add_collection(local_coord)
            if not const_label_created:
                # plot with label
                axes.plot(np.atleast_1d(sv_subset["az_sv_rad"])[-1],
                          np.atleast_1d(sv_subset["el_sv_deg"])[-1],
                          c=color, marker=marker, markersize=8,
                    label=get_label({"gnss_id":constellation}))
                const_label_created = True
            else:
                # plot without label
                axes.plot(np.atleast_1d(sv_subset["az_sv_rad"])[-1],
                          np.atleast_1d(sv_subset["el_sv_deg"])[-1],
                          c=color, marker=marker, markersize=8)
            if add_sv_id_label:
                # offsets move label to the right of marker
                az_offset = 3.*np.radians(np.cos(np.atleast_1d(sv_subset["az_sv_rad"])[-1]))
                el_offset = -3.*np.sin(np.atleast_1d(sv_subset["az_sv_rad"])[-1])
                axes.text(np.atleast_1d(sv_subset["az_sv_rad"])[-1] \
                          + az_offset,
                          np.atleast_1d(sv_subset["el_sv_deg"])[-1] \
                          + el_offset,
                          str(int(sv_name)),
                          )

    # updated axes for skyplot graph specifics
    axes.set_theta_zero_location('N')
    axes.set_theta_direction(-1)
    axes.set_yticks(range(0, 60+10, 30))    # Define the yticks
    axes.set_yticklabels(['',r'$30\degree$',r'$60\degree$'])
    axes.set_ylim(90,0)

    handles, _ = axes.get_legend_handles_labels()
    if len(handles) > 0:
        axes.legend(loc="upper left", bbox_to_anchor=(1.05, 1),
                   title=get_label({"constellation":"constellation"}))

    fig.set_layout_engine(layout='tight')

    if save: # pragma: no cover
        save_figure(fig, "skyplot", prefix=prefix, fnames=fname)
    return fig

In [ ]:
# glp.plot_skyplot()
fig = plot_skyplot(nav_sat, nav_pvt)

fig.savefig(os.path.join(dir_name, "skyplot.png"))

## Correlation between DOP, CN/0, Covariance, hAcc

In [144]:
nav_cov = glp.NavData(csv_path=os.path.join(dir_name, "NAV_COV.csv"))

In [145]:
nav_cov["cov_horiz_rmse"] = np.sqrt(nav_cov["posCovNN"] + nav_cov["posCovEE"])

In [146]:
# may take a few minutes
full_cn0_avg = []
for timestamp, _, subset in glp.loop_time(nav_sat_used,"gps_millis"):
    cn0 = subset["cn0_dbhz"]
    full_cn0_avg.append(np.mean(cn0))

In [ ]:
fig = plt.figure()
plt.plot(nav_pvt["hAcc"]/1000,nav_dop["hDOP"],'o',markersize=1)
plt.xlabel("Horizontal Accuracy Estimate [m]")
plt.ylabel("hDOP")
plt.ylim(0,5)
plt.show()

fig.savefig(os.path.join(dir_name, "hAcc_vs_hDOP.png"))

In [ ]:
fig = plt.figure()
plt.plot(nav_pvt["hAcc"]/1000,full_cn0_avg,'o',markersize=1)
plt.xlabel("Horizontal Accuracy Estimate [m]")
plt.ylabel("C/N0 [dB-Hz]")
plt.show()

fig.savefig(os.path.join(dir_name, "hAcc_vs_cn0.png"))

In [ ]:
fig = plt.figure()
plt.plot(nav_pvt["hAcc"]/1000,nav_cov["cov_horiz_rmse"],'o',markersize=1)
plt.xlabel("Horizontal Accuracy Estimate [m]")
plt.ylabel("Horizontal Covariance RMSE √(cov_nn + cov_ee) [m]")
plt.show()

fig.savefig(os.path.join(dir_name, "hAcc_vs_cov_horiz_rmse.png"))